In [1]:
import pandas as pd
import re
import sys
sys.path.append("../")
from clean import TextCleaner

from transformers import BertPreTrainedModel, BertConfig, BertModel
from transformers import BertTokenizer
import torch
import math
import numpy as np
from patent_utils import PatentCleaner, split_train_test
import gc
from encode import BERTEncoder
from BERTSimilarity import DocumentBert
from torchviz import make_dot

In [2]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("../../data/patent-tsd-matches-11102020.csv")

patent_documents, patent_test = df.patentClaim1.values[:int(0.9 * len(df))], df.patentClaim1.values[int(0.9 * len(df)):]
tsd_documents, tsd_test = df.TSDSection.values[:int(0.9 * len(df))], df.TSDSection.values[int(0.9 * len(df)):]
labels, labels_test = df.match.values[:int(0.9 * len(df))], df.match.values[int(0.9 * len(df)):]

In [3]:
bertEncoder = BERTEncoder(patent_documents, tsd_documents, labels, PatentCleaner())
patent_representations, tsd_representations, correct_output, tsd_dict = bertEncoder.tokenize_train_data()
patent_representations.shape, tsd_representations.shape, correct_output.shape

(torch.Size([2660, 3, 3, 512]),
 torch.Size([2660, 1, 3, 512]),
 torch.Size([2660]))

In [4]:
bertEncoder = BERTEncoder(patent_documents[0])
patent_representations, output = bertEncoder.encode_patents([patent_documents[0]] * len(tsd_representations))

model = torch.load('BERTsimilaritymodel.pth')

patent_representations = patent_representations.to('cpu')
tsd_representations = tsd_representations.to('cpu')
model.to('cpu')

similarity_scores = []
batch_size = 4
gc.collect()
torch.cuda.empty_cache()

for i in range(0, patent_representations.shape[0], batch_size):

    batch_patent_tensors = patent_representations[i:i + batch_size].to(device='cpu')
    batch_tsd_tensors = tsd_representations[i:i + batch_size].to(device='cpu')
#     batch_predictions = model(batch_patent_tensors,
#                               batch_tsd_tensors, 
#                               device='cpu'
#                              )
#     similarity_scores.extend(batch_predictions)
    
# section_scores_dict = {}
# phrases_scores_dict = {}

# for doc_index in tsd_dict.keys():
#     section_scores_dict[doc_index] = predictions[tsd_dict[doc_index]].mean()
#     phrases_scores_dict[doc_index] = predictions[tsd_dict[doc_index]].mean()

In [ ]:
model

In [5]:
batch_predictions = model(batch_patent_tensors,
                              batch_tsd_tensors, 
                              device='cpu'
                             )

C:\ProgramData\Anaconda3\envs\py37cuda10\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [6]:
batch_predictions

tensor([[0.2526],
        [0.9898],
        [0.0180],
        [0.7873]], grad_fn=<SigmoidBackward>)

In [7]:
make_dot(batch_predictions)

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH